# Introduction to tensorflow

This portion of the tutorial, we'll discuss basics of tensorflow in the context of some illustrative numerical examples.

The first step is to import tensorflow and enable eager execution:

In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

tf.executing_eagerly()

print('Loaded TensorFlow version ' + tf.__version__)

Loaded TensorFlow version 1.13.1


After version 1.7, TensorFlow supports a new execution mode that is easier for use with the interactive python interpreter. Google describes its eager execution mode as "an imperative programming environment that evaluates operations immediately, without building graphs: operations return concrete values instead of constructing a computational graph to run later."

TensorFlow used to only support creating a graph first and only evaluating operations in the context of "sessions."

For example (this code constructs a computational graph, but does not initialize variables until Session is started):

```
import tensorflow as tf
import numpy as np

dim1 = 1
dim2 = 1
sigma = 0.1
precision = tf.float32
rho = tf.nn.relu
layers = 3

# placeholders are not compatibe with TF's eager execution mode
x = tf.placeholder(precision, shape = [dim1, None], name = 'input')

with tf.variable_scope('Model'):

    # input layer
    in_W = tf.get_variable(name = 'in_W', shape = [dim2, dim1],
            initializer = tf.random_normal_initializer(stddev = sigma, dtype = precision), 
            dtype = precision)

    in_b = tf.get_variable(name = 'in_b', shape = [dim2, 1],
            initializer = tf.random_normal_initializer(stddev = sigma, dtype = precision), 
            dtype = precision)
    
    x = rho(tf.matmul(in_W, x) + in_b)
    
    # build
    for layer in range(layers):

        W = tf.get_variable(name = 'l' + str(layer) + '_W', shape = [dim1, dim2],
                initializer = tf.random_normal_initializer(stddev = sigma, dtype = precision), 
                dtype = precision)

        b = tf.get_variable(name = 'l' + str(layer) + '_b', shape = [dim2, 1],
                initializer = tf.random_normal_initializer(stddev = sigma, dtype = precision), 
                dtype = precision)
    
        x = rho(tf.matmul(W, x) + b)
        
    # output layer description
    out_v = tf.get_variable(name = 'out_v', shape = [dim1, dim2],
         initializer = tf.random_normal_initializer(stddev = sigma, dtype = precision), 
         dtype = precision)
    
    x = tf.matmul(out_v, x, name = 'output')

with tf.Session() as sess:
    
    # variables such as weights and biases are not actually initialized until here
    sess.run(tf.global_variables_initializer())
    
```

# Basic operations

TF supports basic operations such as addition, squaring, matrix-vector products, etc.

In [5]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))
print(tf.encode_base64("hello world"))

# Operator overloading is also supported
print(tf.square(2) + tf.square(3))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(b'aGVsbG8gd29ybGQ', shape=(), dtype=string)
tf.Tensor(13, shape=(), dtype=int32)


You can determine the shape and data type of a tensor:

In [6]:
x = tf.matmul([[1]], [[2, 3]])
print(x.shape)
print(x.dtype)

(1, 2)
<dtype: 'int32'>


TensorFlow is compatible with python's numpy:

In [7]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)


print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())


TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


Let's try making a tensor with some data. First, we'll create a numpy array. Then we'll convert the numpy array into a TF tensor.

In [8]:
x_data = np.array([[[ 1.,  2.,  3.], [ 4.,  5.,  6.]],
                   [[ 7.,  8.,  9.], [10., 11., 12.]],
                   [[13., 14., 15.], [16., 17., 18.]]])

print(x_data)

print('\n x has shape: \n')

print(np.shape(x_data))

# now let's make a TF tensor from the data
x = tf.convert_to_tensor(x_data, dtype = tf.float32)

print('\n x is now a \n')

print(x)

[[[ 1.  2.  3.]
  [ 4.  5.  6.]]

 [[ 7.  8.  9.]
  [10. 11. 12.]]

 [[13. 14. 15.]
  [16. 17. 18.]]]

 x has shape: 

(3, 2, 3)

 x is now a 

tf.Tensor(
[[[ 1.  2.  3.]
  [ 4.  5.  6.]]

 [[ 7.  8.  9.]
  [10. 11. 12.]]

 [[13. 14. 15.]
  [16. 17. 18.]]], shape=(3, 2, 3), dtype=float32)


# Automatic differentiation

TensorFlow provides the tf.GradientTape API for automatic differentiation - computing the gradient of a computation with respect to its input variables. Tensorflow "records" all operations executed inside the context of a tf.GradientTape onto a "tape". Tensorflow then uses that tape and the gradients associated with each recorded operation to compute the gradients of a "recorded" computation using reverse mode differentiation.

For example:

In [ ]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    # take sum of elements of x
    y = tf.reduce_sum(x)
    print(y)
    # multiply to get y^2
    z = tf.multiply(y, y)
    print(z)

# Derivative of z with respect to the original input tensor x
dz_dx = t.gradient(z, x)
print(dz_dx)
for i in [0, 1]:
    for j in [0, 1]:
        assert dz_dx[i][j].numpy() == 8.0


You can also request gradients of the output with respect to intermediate values computed during a "recorded" tf.GradientTape context.

In [ ]:
x = tf.ones((2, 2))

with tf.GradientTape() as t:
    t.watch(x)
    # take sum of elements of x
    y = tf.reduce_sum(x)
    print(y)
    # multiply to get y^2
    z = tf.multiply(y, y)

# Use the tape to compute the derivative of z with respect to the
# intermediate value y.
dz_dy = t.gradient(z, y)
assert dz_dy.numpy() == 8.0


Slicing is a common operation performed on tensors. Let's try looking at a few slices of x

In [3]:
print('Slicing is performed from a start index to a finishing index.')
print('If I slice from (0,0,0) to (3,2,3), I obtain the entire tensor: \n')
print(tf.slice(x,[0,0,0],[3,2,3])) # the whole tensor

print('\n If I slice from (1,0,0) to (2,2,3), I obtain: \n')
print(tf.slice(x,[1,0,0],[2,2,3]))

print('\n If I slice from (2,0,0) to (3,2,3), I obtain: \n')
print(tf.slice(x,[2,0,0],[1,2,3]))

print('\n If I slice from (0,1,0) to (3,2,3), I obtain: \n')
print(tf.slice(x,[0,1,0],[3,1,3]))
print(tf.slice(x,[0,0,1],[3,2,1]))
print(tf.slice(x,[0,0,1],[3,2,2]))

Slicing is performed from a start index to a finishing index.
If I slice from (0,0,0) to (3,2,3), I obtain the entire tensor: 

tf.Tensor(
[[[ 1.  2.  3.]
  [ 4.  5.  6.]]

 [[ 7.  8.  9.]
  [10. 11. 12.]]

 [[13. 14. 15.]
  [16. 17. 99.]]], shape=(3, 2, 3), dtype=float32)

 If I slice from (1,0,0) to (2,2,3), I obtain: 

tf.Tensor(
[[[ 7.  8.  9.]
  [10. 11. 12.]]

 [[13. 14. 15.]
  [16. 17. 99.]]], shape=(2, 2, 3), dtype=float32)

 If I slice from (2,0,0) to (3,2,3), I obtain: 

tf.Tensor(
[[[13. 14. 15.]
  [16. 17. 99.]]], shape=(1, 2, 3), dtype=float32)

 If I slice from (0,1,0) to (3,2,3), I obtain: 

tf.Tensor(
[[[ 4.  5.  6.]]

 [[10. 11. 12.]]

 [[16. 17. 99.]]], shape=(3, 1, 3), dtype=float32)
tf.Tensor(
[[[ 2.]
  [ 5.]]

 [[ 8.]
  [11.]]

 [[14.]
  [17.]]], shape=(3, 2, 1), dtype=float32)
tf.Tensor(
[[[ 2.  3.]
  [ 5.  6.]]

 [[ 8.  9.]
  [11. 12.]]

 [[14. 15.]
  [17. 99.]]], shape=(3, 2, 2), dtype=float32)
